## Convolutional Neural Network for MNIST image classficiation 

In [1]:
import numpy as np
# from sklearn.utils.extmath import softmax
from matplotlib import pyplot as plt
import re
from tqdm import trange
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
from sklearn.datasets import fetch_openml

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [7]:
import sys
sys.path.append("../")
from src.CNN import CNN, compute_accuracy_metrics, multiclass_accuracy_metrics, list2onehot, onehot2list

In [3]:
import random as random

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
# X = X.values  ### Uncomment this line if you are having type errors in plotting. It is loading as a pandas dataframe, but our indexing is for numpy array. 
X = X / 255.

#since the datasize is too big, I am just going to subsample them. 

#randomlist = random.sample(range(0, 70000), 1500)

#X=X[randomlist]
#y=y[randomlist]
print('X.shape', X.shape)
print('y.shape', y.shape)

'''
Each row of X is a vectroization of an image of 28 x 28 = 784 pixels.  
The corresponding row of y holds the true class label from {0,1, .. , 9}.
'''

X.shape (70000, 784)
y.shape (70000,)


'\nEach row of X is a vectroization of an image of 28 x 28 = 784 pixels.  \nThe corresponding row of y holds the true class label from {0,1, .. , 9}.\n'

In [4]:
def random_padding(img, thickness=1):
    # img = a x b image 
    [a,b] = img.shape
    Y = np.zeros(shape=[a+thickness, b+thickness])
    r_loc = np.random.choice(np.arange(thickness+1))
    c_loc = np.random.choice(np.arange(thickness+1))
    Y[r_loc:r_loc+a, c_loc:c_loc+b] = img
    return Y

In [5]:
def sample_multiclass_MNIST_padding(list_digits=['0','1', '2'], full_MNIST=[X,y], padding_thickness=10):
    # get train and test set from MNIST of given digits
    # e.g., list_digits = ['0', '1', '2']
    # pad each 28 x 28 image with zeros so that it has now "padding_thickness" more rows and columns
    # The original image is superimposed at a uniformly chosen location 
    if full_MNIST is not None:
        X, y = full_MNIST
    else:
        X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
        X = X / 255.
    Y = list2onehot(y.tolist(), list_digits)
    
    idx = [i for i in np.arange(len(y)) if y[i] in list_digits] # list of indices where the label y is in list_digits
    
    X01 = X[idx,:]
    y01 = Y[idx,:]

    X_train = []
    X_test = []
    y_test = [] # list of one-hot encodings (indicator vectors) of each label  
    y_train = [] # list of one-hot encodings (indicator vectors) of each label  

    for i in trange(X01.shape[0]):
        # for each example i, make it into train set with probabiliy 0.8 and into test set otherwise 
        U = np.random.rand() # Uniform([0,1]) variable
        img_padded = random_padding(X01[i,:].reshape(28,28), thickness=padding_thickness)
        img_padded_vec = img_padded.reshape(1,-1)
        if U<0.8:
            X_train.append(img_padded_vec[0,:].copy())
            y_train.append(y01[i,:].copy())
        else:
            X_test.append(img_padded_vec[0,:].copy())
            y_test.append(y01[i,:].copy())

    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    return X_train, X_test, y_train, y_test

In [8]:
# Simple MNIST binary classification experiments 

X_train, X_test, y_train, y_test = sample_multiclass_MNIST_padding(list_digits=['0','1'], 
                                                                   full_MNIST=[X,y],
                                                                   padding_thickness=10)

# data subsampling 
train_size = 100

idx = np.random.choice(np.arange(len(y_train)), train_size)
X_train0 = X_train[idx, :]/np.max(X_train)
y_train0 = y_train[idx, :]

# preprocessing 
out = []
# populate the tuple list with the data
for i in range(X_train0.shape[0]):
    item = list((X_train0[i,:].reshape(1,28+10,28+10), y_train0[i,:])) 
    out.append(item)
    
# FFNN training
CNN0 = CNN(training_data = out,
           f = 5, # conv filter dim
           f_pool = 2, # maxpool filter dim
           num_filt1 = 8, # num filters for the first conv layer
           num_filt2 = 8, # num filters for the second conv layer
           conv_stride = 1,
           pool_stride = 2,
           hidden_nodes = 128)

CNN0.train(lr = 0.01,
           beta1 = 0.95,
           beta2 = 0.99,
           minibatch_size = 32,
           num_epochs = 10,
           verbose = True)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

self.img_x_dim 38
LR:0.01, MiniBatch Size:32


ValueError: too many values to unpack (expected 2)

In [ ]:
print(y_test[0,:])

In [ ]:
X_test /= np.max(X_test)
print('X_test.shape', X_test.shape)
out_test = []
index = [2]

for i in index:
    out_test.append((X_test[i,:].reshape(1,28+10,28+10)))

                
y_hat,conv1,conv2 = CNN0.predict(image_list=out_test)
print(conv1.shape)
print(conv2.shape)

In [ ]:
plt.imshow(conv2[5].reshape(30,30))

In [ ]:
plt.imshow(conv1[1].reshape(24,24))

In [ ]:
plt.imshow(conv1[2].reshape(24,24))

In [ ]:
plt.imshow(conv1[3].reshape(24,24))